In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.utils import shuffle
from torch import autograd
from torch.autograd import Variable

In [2]:
reading = np.load('reading_np.npy')
phn = np.load('phone_np.npy')
drinking = np.load('drinking_np.npy')

In [3]:
y_reading = np.full(50,0)
y_phn = np.full(50,1)
y_drink = np.full(50,2)
Y = np.concatenate((y_reading, y_phn, y_drink), axis = 0)

In [4]:
N = Y.shape[0]

N_actions = 3

dtype = torch.FloatTensor

In [5]:
one_hot_labels = np.zeros((N, N_actions))
one_hot_labels[np.arange(N), Y] = 1

In [6]:
X = np.vstack((phn,reading,drinking))


In [7]:
s = np.arange(X.shape[0])
np.random.shuffle(s)
shuffled_x = X[s]
shuffled_y = Y[s]
train_x = shuffled_x[0:int(0.8*N)]
train_y = shuffled_y[0:int(0.8*N)]
test_x = shuffled_x[int(0.8*N):]
test_y = shuffled_y[int(0.8*N):]

In [8]:
train_x = Variable(torch.from_numpy(train_x)).float()
labels = Variable(torch.from_numpy(train_y))

In [9]:
train_x.shape

torch.Size([120, 273, 20, 3])

In [10]:
class ActionNet(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, label_size, batch_size):
        super(ActionNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden2label = nn.Linear(SEQ_LENGTH*hidden_dim, label_size)
        self.hidden = self.init_hidden()
        print(self.hidden2label)

    def init_hidden(self):
        # the first is the hidden h
        # the second is the cell  c
        return (autograd.Variable(torch.zeros(1, self.batch_size, self.hidden_dim)).cuda(),
                autograd.Variable(torch.zeros(1, self.batch_size, self.hidden_dim)).cuda())

    def forward(self, x):
        #print(x.shape)
        lstm_out, self.hidden = self.lstm(x, self.hidden)
        y  = self.hidden2label(lstm_out.view(BATCH_SIZE,-1))
        log_probs = F.softmax(y, dim=1)
        return y


In [11]:
EMBEDDING_DIM = 60
HIDDEN_DIM = 32
SEQ_LENGTH = train_x.shape[1]
LABEL_SIZE = 3
BATCH_SIZE = 30
num_batches = train_x.shape[0]/BATCH_SIZE
net = ActionNet(embedding_dim=EMBEDDING_DIM, hidden_dim=HIDDEN_DIM, label_size=LABEL_SIZE, batch_size=BATCH_SIZE)

net = net.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr = 0.01, momentum=0.9)

Linear(in_features=8736, out_features=3)


In [12]:
num_epochs = 100
for epoch in range(num_epochs):
    running_loss = 0
    for batch in range(int(num_batches)):
        x = train_x[BATCH_SIZE*batch:BATCH_SIZE*(batch+1)].view(273,BATCH_SIZE,-1).cuda()
        optimizer.zero_grad()
        net.hidden = net.init_hidden()
        
        out = net(x)
        #print(out.shape)
        loss = criterion(out,labels[BATCH_SIZE*batch:BATCH_SIZE*(batch+1)].cuda())
        running_loss += loss.data[0]
        loss.backward()
        optimizer.step()
        
    print ('Epoch [%d/%d], Loss: %.4f' %(epoch+1, num_epochs, running_loss))

Epoch [1/100], Loss: 4.6218
Epoch [2/100], Loss: 5.3488
Epoch [3/100], Loss: 11.3763
Epoch [4/100], Loss: 5.8903
Epoch [5/100], Loss: 3.6253
Epoch [6/100], Loss: 3.7103
Epoch [7/100], Loss: 3.5257
Epoch [8/100], Loss: 2.6290
Epoch [9/100], Loss: 2.5299
Epoch [10/100], Loss: 2.2584
Epoch [11/100], Loss: 1.7973
Epoch [12/100], Loss: 1.6697
Epoch [13/100], Loss: 1.8761
Epoch [14/100], Loss: 2.5706
Epoch [15/100], Loss: 2.4260
Epoch [16/100], Loss: 2.4997
Epoch [17/100], Loss: 3.1274
Epoch [18/100], Loss: 1.8525
Epoch [19/100], Loss: 1.6573
Epoch [20/100], Loss: 1.8858
Epoch [21/100], Loss: 1.2382
Epoch [22/100], Loss: 1.3006
Epoch [23/100], Loss: 1.2258
Epoch [24/100], Loss: 1.0344
Epoch [25/100], Loss: 0.8760
Epoch [26/100], Loss: 0.9636
Epoch [27/100], Loss: 0.8715
Epoch [28/100], Loss: 0.7726
Epoch [29/100], Loss: 0.7453
Epoch [30/100], Loss: 0.7555
Epoch [31/100], Loss: 0.7134
Epoch [32/100], Loss: 0.6805
Epoch [33/100], Loss: 0.7292
Epoch [34/100], Loss: 0.9030
Epoch [35/100], Loss: 

In [13]:
def pred(x):
    out = net(x)
    weight, action = torch.max(out, dim=1)
    return action.data.cpu().numpy()

In [14]:
def accuracy(x,labels):
    N = labels.shape[0]
    predicted = pred(x.cuda())
    i = 0
    for yhat,y in zip(predicted,labels):
        
        if yhat == y:
            i += 1
    print('Accuracy:', i*100/N)

In [15]:
test_x = torch.from_numpy(test_x)

In [16]:

print(test_x.shape)
test_x = test_x.view(SEQ_LENGTH,30,EMBEDDING_DIM)
test_x = Variable(test_x)

torch.Size([30, 273, 20, 3])


In [17]:
pred(test_x.float().cuda())

array([0, 1, 0, 1, 2, 2, 2, 1, 1, 1, 0, 2, 0, 2, 2, 2, 0, 1, 2, 2, 2, 2,
       0, 2, 1, 1, 0, 1, 1, 0])

In [18]:
accuracy(test_x.float().cuda(), test_y)

Accuracy: 96.66666666666667
